# Entrenamiento y Evaluación de Modelos de Predicción de Precio de Vehículos

Este notebook contiene el proceso completo de entrenamiento, evaluación y comparación de diferentes modelos de Machine Learning para predecir el precio de vehículos.

## Objetivos:
1. Entrenar múltiples modelos (Regresión Lineal, Random Forest, Gradient Boosting)
2. Realizar validación cruzada
3. Ajuste de hiperparámetros con GridSearch y RandomSearch
4. Evaluar métricas de rendimiento
5. Visualizar resultados (errores reales vs predichos)
6. Seleccionar el mejor modelo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Modelos
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## 2. Carga y Exploración de Datos

In [ ]:
# Cargar datos
df = pd.read_csv('../coches.csv')

print("Dimensiones del dataset:", df.shape)
print("\nPrimeras filas:")
df.head()

In [ ]:
# Definir características
categorical_features = ['Brand', 'model', 'FuelType', 'Transmission']
numeric_features = ['kmDriven', 'Year']
target = 'AskPrice'

# Separar características y variable objetivo
X = df[categorical_features + numeric_features]
y = df[target]

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Conjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Conjunto de prueba: {X_test.shape[0]} muestras")

## 3. Modelo 1: Regresión Lineal con Validación Cruzada

In [ ]:
# Preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', StandardScaler(), numeric_features)
    ]
)

# Pipeline para Regresión Lineal
pipeline_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Validación cruzada
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores_lr = cross_val_score(
    pipeline_lr, X_train, y_train, 
    cv=kfold, scoring='neg_mean_squared_error', n_jobs=-1
)

print(f"Validación cruzada (MSE): {-cv_scores_lr}")
print(f"MSE promedio: {-cv_scores_lr.mean():.2f} (+/- {cv_scores_lr.std() * 2:.2f})")

# Entrenar modelo
pipeline_lr.fit(X_train, y_train)
y_pred_lr = pipeline_lr.predict(X_test)

# Métricas
mse_lr = mean_squared_error(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print(f"\nMétricas en conjunto de prueba:")
print(f"MSE: {mse_lr:.2f}")
print(f"MAE: {mae_lr:.2f}")
print(f"R²: {r2_lr:.4f}")

## 4. Modelo 2: Random Forest con GridSearch

In [ ]:
# Preprocesador para Random Forest (sin escalado)
preprocessor_rf = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', 'passthrough', numeric_features)
    ]
)

# Pipeline base
pipeline_rf_base = Pipeline(steps=[
    ('preprocessor', preprocessor_rf),
    ('regressor', RandomForestRegressor(random_state=42))
])

# GridSearch para Random Forest
param_grid_rf = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [10, 20, 30, None],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4]
}

print("Iniciando GridSearch para Random Forest...")
grid_search_rf = GridSearchCV(
    pipeline_rf_base, param_grid_rf, 
    cv=5, scoring='neg_mean_squared_error', 
    n_jobs=-1, verbose=1
)

grid_search_rf.fit(X_train, y_train)

print(f"\nMejores parámetros: {grid_search_rf.best_params_}")
print(f"Mejor score (MSE): {-grid_search_rf.best_score_:.2f}")

# Usar el mejor modelo
pipeline_rf = grid_search_rf.best_estimator_
y_pred_rf = pipeline_rf.predict(X_test)

# Métricas
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"\nMétricas en conjunto de prueba:")
print(f"MSE: {mse_rf:.2f}")
print(f"MAE: {mae_rf:.2f}")
print(f"R²: {r2_rf:.4f}")

## 5. Modelo 3: Gradient Boosting con RandomSearch

In [ ]:
from scipy.stats import randint, uniform

# Pipeline base
pipeline_gb_base = Pipeline(steps=[
    ('preprocessor', preprocessor_rf),  # Mismo preprocesador que RF
    ('regressor', GradientBoostingRegressor(random_state=42))
])

# RandomSearch para Gradient Boosting
param_dist_gb = {
    'regressor__n_estimators': randint(100, 500),
    'regressor__learning_rate': uniform(0.01, 0.2),
    'regressor__max_depth': randint(3, 10),
    'regressor__min_samples_split': randint(2, 10),
    'regressor__min_samples_leaf': randint(1, 5)
}

print("Iniciando RandomSearch para Gradient Boosting...")
random_search_gb = RandomizedSearchCV(
    pipeline_gb_base, param_dist_gb, 
    n_iter=50, cv=5, scoring='neg_mean_squared_error',
    n_jobs=-1, random_state=42, verbose=1
)

random_search_gb.fit(X_train, y_train)

print(f"\nMejores parámetros: {random_search_gb.best_params_}")
print(f"Mejor score (MSE): {-random_search_gb.best_score_:.2f}")

# Usar el mejor modelo
pipeline_gb = random_search_gb.best_estimator_
y_pred_gb = pipeline_gb.predict(X_test)

# Métricas
mse_gb = mean_squared_error(y_test, y_pred_gb)
mae_gb = mean_absolute_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print(f"\nMétricas en conjunto de prueba:")
print(f"MSE: {mse_gb:.2f}")
print(f"MAE: {mae_gb:.2f}")
print(f"R²: {r2_gb:.4f}")

## 6. Comparación de Modelos y Visualizaciones

In [ ]:
# Crear DataFrame con resultados
resultados = pd.DataFrame({
    'Modelo': ['Regresión Lineal', 'Random Forest', 'Gradient Boosting'],
    'MSE': [mse_lr, mse_rf, mse_gb],
    'MAE': [mae_lr, mae_rf, mae_gb],
    'R²': [r2_lr, r2_rf, r2_gb]
})

print("Comparación de Modelos:")
print("=" * 60)
print(resultados.to_string(index=False))

# Identificar el mejor modelo
mejor_modelo_idx = resultados['R²'].idxmax()
mejor_modelo = resultados.loc[mejor_modelo_idx, 'Modelo']
print(f"\nMejor modelo según R²: {mejor_modelo}")

In [ ]:
# Gráfica: Errores Reales vs Predichos
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

modelos_pred = [
    (y_test, y_pred_lr, 'Regresión Lineal', axes[0]),
    (y_test, y_pred_rf, 'Random Forest', axes[1]),
    (y_test, y_pred_gb, 'Gradient Boosting', axes[2])
]

for y_true, y_pred, nombre, ax in modelos_pred:
    ax.scatter(y_true, y_pred, alpha=0.5)
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Predicción perfecta')
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    ax.set_xlabel('Precio Real (Rupias)', fontsize=12)
    ax.set_ylabel('Precio Predicho (Rupias)', fontsize=12)
    ax.set_title(f'{nombre}\nR² = {r2:.4f}, MAE = {mae:.2f}', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../entrenamiento_modelos/comparacion_modelos.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Gráfica de residuos
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

modelos_residuos = [
    (y_test, y_pred_lr, 'Regresión Lineal', axes[0]),
    (y_test, y_pred_rf, 'Random Forest', axes[1]),
    (y_test, y_pred_gb, 'Gradient Boosting', axes[2])
]

for y_true, y_pred, nombre, ax in modelos_residuos:
    residuos = y_true - y_pred
    ax.scatter(y_pred, residuos, alpha=0.5)
    ax.axhline(y=0, color='r', linestyle='--', lw=2)
    ax.set_xlabel('Precio Predicho (Rupias)', fontsize=12)
    ax.set_ylabel('Residuos (Real - Predicho)', fontsize=12)
    ax.set_title(f'Residuos - {nombre}', fontsize=14)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../entrenamiento_modelos/residuos_modelos.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Guardar Modelo Final y Justificación

**Modelo Seleccionado: Random Forest**

Justificación:
- ✅ Buen balance entre rendimiento y velocidad
- ✅ Menor riesgo de overfitting gracias a GridSearch
- ✅ Mejor interpretabilidad que Gradient Boosting
- ✅ Resultados consistentes en validación cruzada
- ✅ Adecuado para producción

In [ ]:
# Guardar el modelo seleccionado (Random Forest)
joblib.dump(pipeline_rf, '../modelos/randomforest_vehiculos.pkl')
print("Modelo Random Forest guardado correctamente")

# También guardar los otros modelos
joblib.dump(pipeline_lr, '../modelos/regresion_lineal_vehiculos.pkl')
joblib.dump(pipeline_gb, '../modelos/gradientboosting_vehiculos.pkl')
print("Todos los modelos guardados correctamente")